In [1]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [1]:
from datasets import load_dataset
dataset = load_dataset("Yuhthe/vietnews")

In [6]:
dataset["test"][0]

{'guid': 1,
 'title': 'Bản án cho đối tượng giả danh công an để lừa đảo',
 'abstract': 'Ngày 25/2 , TAND TP. Đà Nẵng tuyên phạt Hồ Xuân Huy ( SN 1994 ) , ngụ quận Hải Châu , 12 năm tù về tội Lừa đảo chiếm đoạt tài sản .',
 'article': 'Theo lời khai của Huy tại phiên toà , để có tiền sử dụng cá nhân , Huy " nổ " là sĩ quan cục Phòng chống ma tuý của bộ Công an đóng tại TP. Đà Nẵng , có nguồn mua ô tô thanh lý giá rẻ , và khả năng chạy việc vào ngành công an . Chỉ với lời " nổ " này , từ tháng 10/2016 đến 9/2017 , nhiều người đã bị lừa đảo với tổng số tiền 3,2 tỷ đồng . Trong đó , người bị Huy lừa nhiều nhất là vợ chồng ông Bảo Th . , ngụ quận Hải Châu . Huy giới thiệu với cặp vợ chồng này mình có suất mua ô tô thanh lý giá rẻ và rủ họ mua cùng . Tin lời , vợ chồng ông Th . đưa cho Huy hơn 1 tỷ đồng . Cùng thủ đoạn , Huy lừa thêm ông Nguyễn Tấn T. 970 triệu đồng , Lê Quốc Th . 400 triệu đồng , Trần Nhật S. 300 triệu đồng … Sau chiêu thức mua xe thanh lý , Huy chuyển sang giả vờ có khả nă

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'abstract', 'article'],
        num_rows: 99134
    })
    validation: Dataset({
        features: ['guid', 'title', 'abstract', 'article'],
        num_rows: 22184
    })
    test: Dataset({
        features: ['guid', 'title', 'abstract', 'article'],
        num_rows: 22498
    })
})

In [4]:
#dataset["train"]["abstract"]

In [5]:
# df = pd.DataFrame({"prompt": texts, "output": labels}) 
# df.to_csv("short_texts/dataset.csv", index=False)
# dataset = Dataset.from_dict(df)


In [6]:
train_dataset = dataset["train"]
validation_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [7]:
from datasets import Dataset

# Assuming `dataset` is your Dataset object
train_dataset = train_dataset.map(lambda example: {'text':  example['article']  + "###Tóm tắt: "+ example['abstract'] +"***END"})
validation_dataset = validation_dataset.map(lambda example: {'text':  example['article']  + "###Tóm tắt: "+ example['abstract'] +"***END"})
test_dataset = test_dataset.map(lambda example: {'text':  example['article']  + "###Tóm tắt: "+ example['abstract'] +"***END"})


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

# model_name = "TinyPixel/Llama-2-7B-bf16-sharded"
# model_name = "vilm/vinallama-2.7b-chat"
# model_name = "vinai/PhoGPT-7B5-Instruct"
model_name = "bloom560m/checkpoint-100000"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
# )

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=torch.float16,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [9]:
from transformers import TrainingArguments

from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64


peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)


output_dir = "./bloom560m"
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 10000
num_train_epochs = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 200000
warmup_ratio = 0.03
lr_scheduler_type = "linear"


training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [10]:
from trl import SFTTrainer

max_seq_length = 1568


trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset = validation_dataset,
    # peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

# for name, module in trainer.model.named_modules():
#     if "norm" in name:
#         module = module.to(torch.float32)

Map:   0%|          | 0/99134 [00:00<?, ? examples/s]

Map:   0%|          | 0/22184 [00:00<?, ? examples/s]

/home/inresai/anaconda3/envs/llminresai/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:282: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ndtoan-10a10kx (inresai). Use `wandb login --relogin` to force relogin


  0%|          | 0/200000 [00:00<?, ?it/s]

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.876, 'learning_rate': 3.3333333333333335e-07, 'epoch': 0.0}
{'loss': 1.7192, 'learning_rate': 6.666666666666667e-07, 'epoch': 0.0}
{'loss': 1.5737, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 1.6274, 'learning_rate': 1.3333333333333334e-06, 'epoch': 0.0}
{'loss': 1.6062, 'learning_rate': 1.6666666666666667e-06, 'epoch': 0.0}
{'loss': 2.0034, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}
{'loss': 1.6756, 'learning_rate': 2.3333333333333336e-06, 'epoch': 0.0}
{'loss': 1.79, 'learning_rate': 2.666666666666667e-06, 'epoch': 0.0}
{'loss': 1.5209, 'learning_rate': 3e-06, 'epoch': 0.0}
{'loss': 1.4913, 'learning_rate': 3.3333333333333333e-06, 'epoch': 0.0}
{'loss': 2.2047, 'learning_rate': 3.666666666666667e-06, 'epoch': 0.0}
{'loss': 1.8213, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 1.529, 'learning_rate': 4.333333333333334e-06, 'epoch': 0.0}
{'loss': 1.4188, 'learning_rate': 4.666666666666667e-06, 'epoch': 0.0}
{'loss': 1.3678, 'l

wandb: Network error (ConnectionError), entering retry loop.


{'loss': 1.994, 'learning_rate': 0.0001944577319587629, 'epoch': 0.23}
{'loss': 1.9302, 'learning_rate': 0.0001944474226804124, 'epoch': 0.23}
{'loss': 1.6179, 'learning_rate': 0.00019443711340206185, 'epoch': 0.23}
{'loss': 2.1772, 'learning_rate': 0.00019442680412371134, 'epoch': 0.23}
{'loss': 2.0338, 'learning_rate': 0.00019441649484536083, 'epoch': 0.23}
{'loss': 2.0716, 'learning_rate': 0.00019440618556701032, 'epoch': 0.23}
{'loss': 1.7704, 'learning_rate': 0.0001943958762886598, 'epoch': 0.23}
{'loss': 1.6801, 'learning_rate': 0.00019438556701030927, 'epoch': 0.23}
{'loss': 2.187, 'learning_rate': 0.00019437525773195876, 'epoch': 0.23}
{'loss': 2.1978, 'learning_rate': 0.00019436494845360825, 'epoch': 0.23}
{'loss': 1.8747, 'learning_rate': 0.00019435463917525774, 'epoch': 0.23}
{'loss': 1.802, 'learning_rate': 0.00019434432989690723, 'epoch': 0.23}
{'loss': 1.5313, 'learning_rate': 0.0001943340206185567, 'epoch': 0.23}
{'loss': 2.2247, 'learning_rate': 0.00019432371134020618, 

TrainOutput(global_step=200000, training_loss=1.5283703484427928, metrics={'train_runtime': 203026.2036, 'train_samples_per_second': 1.97, 'train_steps_per_second': 0.985, 'train_loss': 1.5283703484427928, 'epoch': 4.03})

In [12]:
# model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
# model_to_save = trainer.model
model.save_pretrained("outputs")


In [13]:
# lora_config = LoraConfig.from_pretrained('results/checkpoint-5000')
# model = get_peft_model(model, lora_config)


In [14]:
text = validation_dataset[3]["article"]
text

'Vào khoảng 15h ngày 24/7 , Thạch Sà Kh . ( SN 1983 , ngụ xã Hưng Hội , huyện Vĩnh Lợi , tỉnh Bạc Liêu ) nghi bị " ngáo đá " bất ngờ cầm dao và một khúc gỗ truy sát hàng chục người tại ấp Đay Tà Ni , xã Hưng Hội , huyện Vĩnh Lợi . Vụ việc khiến 11 người thương vong . Nhận được tin báo , lực lượng công an nhanh chóng có mặt tại hiện trường , khống chế và bắt giữ Kh .. Đồng thời , lực lượng chức năng đã đưa người bị thương đến trạm Y tế xã Hưng Hội và bệnh viện Đa khoa Bạc Liêu cấp cứu . Đến 17h chiều cùng ngày , vụ truy sát đã có 1 người tử vong . Nạn nhân là bà Hồ Thị Tr . ( 81 tuổi ) . Ngoài ra , vụ việc còn khiến 10 người khác bị thương đang phải điều trị tích cực tại bệnh viện Đa khoa Bạc Liêu . Theo lãnh đạo xã Hưng Hội , trước đây , Kh . là người ở địa phương . Đối tượng đã có vợ con . Sau đó , Kh . đến sinh sống ở tỉnh Trà Vinh và mới về xã Hưng Hội cách đây khoảng vài ngày thì gây ra vụ việc đáng tiếc trên . Hiện , nguyên nhân vụ việc đang được cơ quan chức năng điều tra , làm r

In [15]:
# text = """
# TP.HCM cần 43 giáo viên từ sinh viên xuất sắc, bao nhiêu ứng viên trúng tuyển?.
# Đợt tuyển dụng giáo viên cho năm học 2023-2024 của Sở GD-ĐT TP.HCM diễn ra vào tháng 6.2023. Theo đó, căn cứ vào quyết định phê duyệt của Chủ tịch UBND TP.HCM về kết quả tuyển dụng viên chức thành phố từ nguồn sinh viên xuất sắc, cán bộ khoa học trẻ năm 2023, Sở GD-ĐT TP.HCM đã có thông báo kết quả tuyển giáo viên năm 2023. Cụ thể, có 2 sinh viên xuất sắc chuyên ngành sư phạm toán và sư phạm tiếng Anh đã trúng tuyển theo hình thức tuyển thẳng vị trí giáo viên THPT. Với kết quả trên, 2 ứng viên trúng tuyển sẽ hoàn thiện hồ sơ tuyển dụng trong vòng 30 ngày sau đó Sở GD-ĐT sẽ phân công nhiệm sở.
# Được biết, đây là lần đầu tiên Sở GD-ĐT TP.HCM xét tuyển thẳng giáo viên từ nguồn sinh viên tốt nghiệp xuất sắc, cán bộ khoa học trẻ. Được biết tổng nhu cầu tuyển dụng viên chức trong các đơn vị giáo dục công lập từ nguồn sinh viên tốt nghiệp xuất sắc là 43 chỉ tiêu các chuyên ngành sư phạm: tin học, tiếng Anh, âm nhạc, mỹ thuật, toán học, vật lý, hóa học, sinh học, ngữ văn, lịch sử, địa lý, giáo dục thể chất, giáo dục quốc phòng-an ninh.
# Theo đánh giá của một cán bộ phụ trách công tác tổ chức cán bộ của Sở GD-ĐT, số lượng ứng viên đủ điều kiện xét tuyển theo chính sách này còn hạn chế do ràng buộc nhiều quy định. Chẳng hạn như ứng viên tốt nghiệp ĐH loại xuất sắc nhưng chỉ cần một học kỳ không đạt cũng trượt vòng sơ tuyển. Ngoài ra, trong thời gian học cấp THPT, ứng viên phải có học lực giỏi, đạt giải ba trở lên tại các kỳ thi học sinh giỏi cấp tỉnh, quốc gia…
# """

In [16]:
dataset[3]["abstract"]

KeyError: "Invalid key: 3. Please first select a split. For example: `my_dataset_dictionary['train'][3]`. Available splits: ['test', 'train', 'validation']"

In [ ]:

device = "cuda:0"

prompt = text
inputs = tokenizer(prompt, return_token_type_ids = False, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=300, top_k=50,  top_p=1, temperature = 0.8)
output = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = output.split("###Tóm tắt:")[1]
print(response)

/home/inresai/anaconda3/envs/llminresai/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


 Sở GD-ĐT TP.HCM đang đối mặt với tình trạng tuyển dụng viên chức và sinh viên không đạt yêu cầu. Đã có 382 thí sinh tham gia so với năm ngoái, trong đó có 382 thí sinh dự thi học sinh giỏi cấp quốc gia. Sở GD-ĐT đang đối mặt với tình trạng tuyển dụng viên chức và sinh viên không đạt yêu cầu. Cơ quan GD-ĐT TP.HCM cũng đang xem xét việc tuyển dụng viên chức của các thí sinh và quy định rằng một số giải pháp có thể xuất hiện trong tương lai.***END


In [ ]:
#format phobert
# device = "cuda:0"
# PROMPT_TEMPLATE = "### Câu hỏi:\n{instruction}\n\n### Trả lời:" 

# instruction = "Tóm tắt đoạn trên: Nghi vấn người đóng giả gấu ở vườn thú Trung Quốc: Nhận định của chuyên gia Du khách Trung Quốc từ khắp nơi đổ xô tới vườn thú ở thành phố Hàng Châu, tỉnh Chiết Giang để xem tận mắt con gấu chó bị nghi là nhân viên vườn thú đóng giả. Trang chủ Thế giới Thứ Tư, ngày 02/08/2023 11:15 AM (GMT+7) Du khách Trung Quốc từ khắp nơi đổ xô tới vườn thú ở thành phố Hàng Châu, tỉnh Chiết Giang để xem tận mắt con gấu chó bị nghi là nhân viên vườn thú đóng giả. Các tin đồn lan truyền sau khi một video do du khách quay lại, cho thấy cảnh con gấu đứng thẳng bằng hai chân và bộ lông trông kỳ lạ, giống như một người hóa trang thành gấu. Du khách Trung Quốc từ nhiều địa phương đã đổ xô đến vườn thú Hàng Châu, ở thành phố cùng tên thuộc tỉnh Chiết Giang, để có thể tận mắt chiêm ngưỡng. Theo tờ Chao News có trụ sở ở Chiết Giang, số lượng du khách đến vườn thú đã tăng 30%, lên tới khoảng 20.000 người/ngày kể từ khi video quay cảnh con gấu chó được lan truyền trên mạng xã hội. Du khách Trung Quốc từ khắp nơi đổ xô tới vườn thú Hàng Châu xem con gấu chó bị nghi là người hóa trang. \"Sau khi xem video con gấu đứng thẳng với dáng vẻ khác lạ trên mạng, tôi tới đây để xem thực sự trông như thế nào\", một người đàn ông họ You nói, theo Chao News. \"Chúng tôi đã đáp chuyến tàu cao tốc từ thành phố Tô Châu tới Hàng Châu để chiêm ngưỡng con gấu\", một du khách khác tên Qian Ming, nói với phóng viên\nNghi vấn người hóa trang thành gấu chó trong sở thú Trung Quốc Sau video quay cảnh gấu chó đứng trên hai chân giống như người, một sở thú ở Trung Quốc phủ nhận tin đồn các con gấu là do người hóa trang. Sau video quay cảnh gấu chó đứng trên hai chân giống như người, một sở thú ở Trung Quốc phủ nhận tin đồn các con gấu là do người hóa trang. Hình ảnh gấu chó do sở thú Hàng Châu cung cấp và gấu chó trong đoạn video được lan truyền - Ảnh chụp màn hìnhTin tức từ CNN hôm 1-8 (giờ địa phương), sở thú Hàng Châu (Trung Quốc) đã bác bỏ ý kiến cho rằng một số gấu chó của họ có thể là người, được hóa trang để lừa du khách.Nghi ngờ không vô cớĐại diện từ vườn thú đã đưa ra một tuyên bố trên fanpage, được viết từ góc nhìn của một con gấu chó tên là Angela để chứng minh việc mọi người không hiểu nhiều về loài vật này.\"Tôi là gấu chó Angela - tôi nhận được một cuộc gọi sau giờ làm việc ngày hôm qua từ người đứng đầu vườn thú hỏi liệu hôm nay tôi có lười biếng, trốn việc và tìm một người thay thế không.Hãy để tôi nhắc lại một lần nữa với mọi người rằng tôi là gấu chó - không phải gấu đen, không phải chó mà là gấu chó\", ban quản lý viết.Trên NBC News, Wong Siew Te - nhà sinh vật học hoang dã sáng lập nên Trung tâm bảo tồn gấu chó Bornean (Malaysia) cho biết con vật trong video chắc chắn là gấu chó. Ông đã nghiên cứu về loài vật này trong"
# # instruction = "Sửa lỗi chính tả:\nTriệt phá băng nhóm kướp ô tô, sử dụng \"vũ khí nóng\""

# input_prompt = PROMPT_TEMPLATE.format_map(  
#     {"instruction": instruction}  
# )  
  
# input_ids = tokenizer(input_prompt, return_tensors="pt")  
  
# outputs = model.generate(  
#     inputs=input_ids["input_ids"].to(device),  
#     attention_mask=input_ids["attention_mask"].to(device),  
#     do_sample=True,  
#     temperature=1.0,  
#     top_k=50,  
#     top_p=0.9,  
#     max_new_tokens=128,  
#     eos_token_id=tokenizer.eos_token_id,  
#     pad_token_id=tokenizer.pad_token_id  
# )  
# response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  
# response = response.split("### Trả lời:")[1]

In [ ]:
response

' Sở GD-ĐT TP.HCM đang đối mặt với tình trạng tuyển dụng viên chức và sinh viên không đạt yêu cầu. Đã có 382 thí sinh tham gia so với năm ngoái, trong đó có 382 thí sinh dự thi học sinh giỏi cấp quốc gia. Sở GD-ĐT đang đối mặt với tình trạng tuyển dụng viên chức và sinh viên không đạt yêu cầu. Cơ quan GD-ĐT TP.HCM cũng đang xem xét việc tuyển dụng viên chức của các thí sinh và quy định rằng một số giải pháp có thể xuất hiện trong tương lai.***END'